In [2]:
########################### Imports and DB Work

In [4]:
import requests
import json

In [5]:
import mysql.connector 

In [6]:
import config

In [7]:
cnx = mysql.connector .connect(
    host = config.host,
    user = config.user,
    passwd = config.pw)


In [8]:
    
cursor = cnx.cursor()

In [13]:
cursor.execute("CREATE DATABASE yelp")

In [12]:
cnx.database = 'yelp'

In [9]:
cursor

In [29]:
create_table_businesses = """CREATE TABLE businesses
                        (business_id VARCHAR(225) PRIMARY KEY,
                        name VARCHAR(225) NOT NULL,
                        address VARCHAR(225) NOT NULL,
                        rating FLOAT NOT NULL,
                        price VARCHAR(50),
                        review_count INT NOT NULL);"""

In [30]:
cursor.execute(create_table_businesses)

In [10]:
create_table_reviews = """CREATE TABLE reviews
                        (review_id VARCHAR(30) PRIMARY KEY,
                        review_text VARCHAR(150) NOT NULL,
                        rating FLOAT,
                        time_created NUMERIC,
                        business_id VARCHAR(225) NOT NULL,
                        CONSTRAINT fk_businesses
                        FOREIGN KEY (business_id)
                            REFERENCES businesses(business_id));"""

In [13]:
cursor.execute(create_table_reviews)

In [ ]:
############################## Yelp call 1 setup

In [38]:
url = 'https://api.yelp.com/v3/businesses/search'

In [39]:
client_id = 'SrP-vVinyIiYU2-NOuqnOA'
api_key = 'CaJ2qY7XSMjW7nhV9IK8KiEYI6Yj3kPAXR6Jdgstzo-YOFuAfT3AbrRF6UID6P9ia75sS8dzlih1HNcZEK1EHzu8Dp-gOnC3eVdyNpHKt4iJIUIOC7ovxCM68jrpXXYx'

In [40]:
headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

In [41]:
# what type of business do you want to search
term = 'bagel'

#where do you want to perform this search
location = 'East Village'

# what is your other parameter you want to search against
radius = 5000

limit = 50

In [43]:
url_params = {
                "term": term.replace(' ', '+'),
                "location": location.replace(' ', '+'),
                "radius" : radius,
                "limit" : limit
            }

In [44]:
response = requests.get(url, headers=headers, params=url_params)

In [ ]:
############################# Working on the response

In [45]:
data = response.json()

In [46]:
data['businesses'][0]

{'id': 'VrCCr45dhN-RRM107iptdg',
 'alias': 'russ-and-daughters-new-york',
 'name': 'Russ & Daughters',
 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/EN-MJ-kuk_-ihX1_DKHgFA/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/russ-and-daughters-new-york?adjust_creative=SrP-vVinyIiYU2-NOuqnOA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=SrP-vVinyIiYU2-NOuqnOA',
 'review_count': 2142,
 'categories': [{'alias': 'bagels', 'title': 'Bagels'},
  {'alias': 'smokehouse', 'title': 'Smokehouse'},
  {'alias': 'bakeries', 'title': 'Bakeries'}],
 'rating': 4.5,
 'coordinates': {'latitude': 40.722589, 'longitude': -73.9882828294139},
 'transactions': [],
 'price': '$$',
 'location': {'address1': '179 E Houston St',
  'address2': '',
  'address3': '',
  'city': 'New York',
  'zip_code': '10002',
  'country': 'US',
  'state': 'NY',
  'display_address': ['179 E Houston St', 'New York, NY 10002']},
 'phone': '+12124754880',
 'display_phone': '(212) 475-4880',
 

In [ ]:
########################### Parsing

In [47]:
business = data['businesses'][0]

In [75]:
#Create a tuple that contains only the information that we want
#the join is used because that result comes back as a list and we want to convert into a string
biz_tuple = (business['id'], business['name'], ' '.join(business['location']['display_address']), business['rating'], len(business['price']), business['review_count'])

In [79]:
biz_tuple

('VrCCr45dhN-RRM107iptdg',
 'Russ & Daughters',
 '179 E Houston St New York, NY 10002',
 4.5,
 2,
 2142)

In [80]:
def parse_data(data_list):
    # create a container to hold our parsed data
    parse_data = []  
    
    # loop through our business and parse each individual business
    for business in data_list:
        try:
            biz_tuple = (business['id'], business['name'], ' '.join(business['location']['display_address']), business['rating'], len(business['price']), business['review_count'])
    # in the loop, add each individual parsed parsed business to our data container
            parse_data.append(biz_tuple)
    # return the container with all of the parsed results     
        except:
            biz_tuple = (business['id'], business['name'], ' '.join(business['location']['display_address']), business['rating'], 0, business['review_count'])
            parse_data.append(biz_tuple)
    return parse_data


In [81]:
parse_data(data['businesses'])

[('VrCCr45dhN-RRM107iptdg',
  'Russ & Daughters',
  '179 E Houston St New York, NY 10002',
  4.5,
  2,
  2142),
 ('j1S3NUrkB3BVT49n_e76NQ',
  'Best Bagel & Coffee',
  '225 W 35th St New York, NY 10001',
  4.5,
  1,
  3166),
 ('oi39VAwo4-KGm_gSkWPCsQ',
  'Tompkins Square Bagels - Avenue A',
  '165 Avenue A New York, NY 10009',
  4.0,
  1,
  1077),
 ('foO2N-TrdPBO-dFn6M35TA',
  'Brooklyn Bagel & Coffee Company',
  '63 E 8th St New York, NY 10003',
  4.5,
  0,
  88),
 ('BIFbVuIDJFS_2u_8vJxnwA',
  'Tompkins Square Bagels -  2nd Ave',
  '184 2nd Ave New York, NY 10003',
  4.0,
  1,
  199),
 ('c3eMI4_o4dPDDhPV_ibBYQ',
  'Ess-a-Bagel',
  '831 3rd Ave New York, NY 10022',
  4.0,
  1,
  3471),
 ('u9zVZH0liQu9skIz1kGrMA',
  "Heaven's Hot Bagel",
  '283 E Houston St New York, NY 10002',
  4.0,
  1,
  192),
 ('SCornvTSCMQ2SvWGq8Z_Mw',
  'Ess-A-Bagel',
  '324 1st Ave New York, NY 10009',
  4.0,
  1,
  845),
 ('gwo5ci1qtZZhOURn81ddhg',
  "Greenberg's Bagels",
  '103 N 3rd St Brooklyn, NY 11249',
  3

In [85]:
parsed_data = parse_data(data['businesses'])

In [ ]:
###################### Inserting

In [ ]:
#def db_insert(cnx, cursor, parsed results):
    # your code to insert and commit the results
    # I would create the connection and cursor outside of this function and then pass it through

In [82]:
stmt = "INSERT INTO businesses (business_id, name, address, rating, price, review_count) VALUES (%s, %s, %s, %s, %s, %s)"

In [86]:
cursor.executemany(stmt, parsed_data)
cnx.commit()

In [ ]:
###################### Variable Tracking

In [ ]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    db_insert(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 50

In [ ]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    response = requests.get(url, headers=headers, params=url_params)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(response)
    
    # use your function to insert your parsed results into the db
    db_insert(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 50